In [10]:
!pip -q install kaggle

In [11]:
import os
import json
import pandas as pd
import re
import numpy as np
import time
import matplotlib.pyplot as plt
import collections
import random
import requests
from math import sqrt
import seaborn as sns
import email
import warnings 
warnings.filterwarnings('ignore')


In [12]:
if not os.path.exists('/content/data/'):

    api_token = {"username": "gomteshjain12",
                 "key": "c1ce67cc586abf35c84ba89756ab5b72"}

    with open('/content/kaggle.json', 'w') as file:
        json.dump(api_token, file)

    os.environ["KAGGLE_CONFIG_DIR"] = "/content/"

    os.system('kaggle datasets download -d wcukierski/enron-email-dataset')
    os.makedirs('/content/data/', exist_ok=True)
    os.system('mv /content/enron-email-dataset.zip /content/data/enron-email-dataset.zip')
    os.system('unzip -q /content/data/enron-email-dataset.zip -d /content/data/')
    os.remove('/content/data/enron-email-dataset.zip')

In [13]:
df= pd.read_csv("/content/data/emails.csv")

# EDA

In [14]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [15]:
# check the shape 
df.shape

(517401, 2)

In [16]:
# get message value
print(df.loc[1]['file'])
print("\n")
print(df.loc[1]['message'])

allen-p/_sent_mail/10.


Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and wh

In [17]:
# transform the email into correct form
message = df.loc[1]['message']
emails = email.message_from_string(message)
emails.items()

[('Message-ID', '<15464986.1075855378456.JavaMail.evans@thyme>'),
 ('Date', 'Fri, 4 May 2001 13:51:00 -0700 (PDT)'),
 ('From', 'phillip.allen@enron.com'),
 ('To', 'john.lavorato@enron.com'),
 ('Subject', 'Re:'),
 ('Mime-Version', '1.0'),
 ('Content-Type', 'text/plain; charset=us-ascii'),
 ('Content-Transfer-Encoding', '7bit'),
 ('X-From', 'Phillip K Allen'),
 ('X-To', 'John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>'),
 ('X-cc', ''),
 ('X-bcc', ''),
 ('X-Folder', "\\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail"),
 ('X-Origin', 'Allen-P'),
 ('X-FileName', 'pallen (Non-Privileged).pst')]

In [18]:
# get email body
emails.get_payload()

"Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  \n\nMy suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.\n"

In [31]:
emails.get("Date")


'Fri, 4 May 2001 13:51:00 -0700 (PDT)'

In [3]:
import re
import spacy
from textblob import TextBlob

nlp = spacy.load('en_core_web_sm')



In [39]:
message = df.loc[9]['message']

In [40]:
# Parse email
msg = email.message_from_string(message)
sender = msg['From']
recipient = msg['To']
subject = msg['Subject']
body = msg.get_payload()

In [41]:
# Tokenize text
doc = nlp(body)

In [42]:
# Extract named entities
dates = []
times = []
places = []
for ent in doc.ents:
    if ent.label_ == 'DATE':
        dates.append(ent.text)
    elif ent.label_ == 'TIME':
        times.append(ent.text)
    elif ent.label_ == 'GPE':
        places.append(ent.text)

In [43]:
# Perform sentiment analysis
blob = TextBlob(body)
sentiment = blob.sentiment.polarity


In [44]:
# Extract important information
action_items = re.findall(r'(?:\n|^)[\*\-\+]\s+(.*?)(?=\n[\*\-\+]|$)', body, re.DOTALL)
deadlines = re.findall(r'(?:by|before|due\s+date)\s+(\w+\s+\d+)', body, re.IGNORECASE)
requests = re.findall(r'(?:can|could|would|please)\s+(.*?)(?=\?)', body)

In [45]:
# Print extracted information
print(f'Sender: {sender}')
print(f'Recipient: {recipient}')
print(f'Subject: {subject}')
print(f'Dates: {dates}')
print(f'Times: {times}')
print(f'Places: {places}')
print(f'Sentiment: {sentiment}')
print(f'Action items: {action_items}')
print(f'Deadlines: {deadlines}')
print(f'Requests: {requests}')

Sender: phillip.allen@enron.com
Recipient: zimam@enron.com
Subject: FW: fixed forward or other Collar floor gas price terms
Dates: ['3', '5', '7', '10 years', 'May through September']
Times: ['10/16/2000 \n01:42 PM', '10/12/2000 01:12:21 PM', '6-8 hours']
Places: ['San Diego', 'P.E.', 'Albuquerque']
Sentiment: 0.0781641604010025
Action items: []
Deadlines: []
Requests: []


In [55]:
def informationEmail(message):

  # Parse email
  msg = email.message_from_string(message)
  sender = msg['From']
  recipient = msg['To']
  subject = msg['Subject']
  body = msg.get_payload()

  # Tokenize text
  doc = nlp(body)

  # Extract named entities
  dates = []
  times = []
  places = []
  for ent in doc.ents:
      if ent.label_ == 'DATE':
          dates.append(ent.text)
      elif ent.label_ == 'TIME':
          times.append(ent.text)
      elif ent.label_ == 'GPE':
          places.append(ent.text)

  # Perform sentiment analysis
  blob = TextBlob(body)
  sentiment = blob.sentiment.polarity
  # Extract important information
  action_items = re.findall(r'(?:\n|^)[\*\-\+]\s+(.*?)(?=\n[\*\-\+]|$)', body, re.DOTALL)
  deadlines = re.findall(r'(?:by|before|due\s+date)\s+(\w+\s+\d+)', body, re.IGNORECASE)
  requests = re.findall(r'(?:can|could|would|please)\s+(.*?)(?=\?)', body)

  #   # Print extracted information
  # print()
  # print(f'Sender: {sender}')
  # print(f'Recipient: {recipient}')
  # print(f'Subject: {subject}')
  # print(f'Dates: {dates}')
  # print(f'Times: {times}')
  # print(f'Places: {places}')
  # print(f'Sentiment: {sentiment}')
  # print(f'Action items: {action_items}')
  # print(f'Deadlines: {deadlines}')
  # print(f'Requests: {requests}')

  # print("-------------------------------")
  # print()
# Create DataFrame for this email
  email_df ={
        'Sender': [sender],
        'Recipient': [recipient],
        'Subject': [subject],
        'Dates': [dates],
        'Times': [times],
        'Places': [places],
        'Sentiment': [sentiment],
        'Action Items': [action_items],
        'Deadlines': [deadlines],
        'Requests': [requests]
    }
  #print(email_df)
  # Append DataFrame to master DataFrame
  #finaldata = pd.concat([finaldata, email_df], ignore_index=True)
  #finaldata = finaldata.append(email_df, ignore_index=True)
  return finaldata.append(email_df, ignore_index=True)



In [56]:
n=len(df['message'])
columns_names=['Sender','Recipient','Subject','Dates','Times','Places','Sentiment','Action Items','Deadlines','Requests']
finaldata = pd.DataFrame(columns =columns_names )
for i in range(0,n):
  message = df.loc[i]['message']
  finaldata = informationEmail(message)

Exception ignored in: <function _xla_gc_callback at 0x7f7cdf8dab00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: ignored

In [59]:
len(df['message'])


517401

In [57]:
finaldata

,Sender,Recipient,Subject,Dates,Times,Places,Sentiment,Action Items,Deadlines,Requests
0,[phillip.allen@enron.com],[tim.belden@enron.com],[],[[]],[[]],[[]],[0.0],[[]],[[]],[[]]
1,[phillip.allen@enron.com],[john.lavorato@enron.com],[Re:],[[]],[[]],[[]],[0.18181818181818182],[[]],[[]],[[]]
2,[phillip.allen@enron.com],[leah.arsdall@enron.com],[Re: test],[[]],[[]],[[]],[1.0],[[]],[[]],[[]]
3,[phillip.allen@enron.com],[randall.gay@enron.com],[],[[]],[[]],[[]],[0.0],[[]],[[]],[[]]
4,[phillip.allen@enron.com],[greg.piper@enron.com],[Re: Hello],[[Tuesday]],[[11:45]],[[]],[0.0],[[]],[[]],[[]]
...,...,...,...,...,...,...,...,...,...,...
95840,[savita.puthigai@enron.com],"[martin.cuilla@enron.com, kevin.ruscitti@enron...",[IMPORTANT New Website Release],[[tomorrow]],[[tonight]],[[]],[0.20995670995670998],[[]],[[]],[[]]
95841,[savita.puthigai@enron.com],[traders.eol@enron.com],[VERY IMPORTANT ENRONONLINE STACK MANAGER CHA...,"[[the next week, Friday, Today, Today]]",[[tonight]],[[]],[0.13025493025493026],[[]],[[Friday 5]],[[]]
95842,[mark.schrab@enron.com],[charlene.merrill@dvn.com],[Re: HIOS],"[[the 28th of this month, the weekend, april 2...","[[this morning, 04/27/2001 10:00:40 AM]]",[[]],[0.23333333333333334],[[]],[[]],[[]]
95843,[40enron@enron.com],[None],[SUPPLEMENTAL Weekend Outage Report for 4-27-0...,"[[April 27, 2001 5:00pm, April 30, 2001 12:00a...","[[6:00:00 PM, 8:00:00 PM, 4:00:00 PM, 6:00:00 ...","[[London, London, London, Contact(s]]",[0.0673076923076923],[[]],[[]],[[]]


In [60]:
finaldata.to_csv('file1.csv')

# Event From Any Text

In [4]:
import spacy

# # Load the English language model
nlp = spacy.load('en_core_web_sm')


In [9]:


# Define the text to analyze
text ="Hey, let's meet at Starbucks at 2pm on Friday to discuss the project."
#"The 5th annual TechCon conference will be held from August 22-24, 2023 in San Francisco, CA. The conference will feature keynote speeches from industry experts, including Elon Musk, Jeff Bezos, and Sundar Pichai."
#"The annual conference on machine learning will take place on May 10-12, 2023 at 10:00 AM at venu the San Francisco Marriott Marquis. Keynote speakers include Andrew Ng and Yoshua Bengio."

# Apply the NLP pipeline to the text
doc = nlp(text)

# Extract relevant information using named entity recognition (NER), part-of-speech (POS) tagging, and dependency parsing
def extract_information(text):
    # Parse text with spaCy
    doc = nlp(text)
    
    # Initialize variables to store information
    place = []
    event = []
    time = []
    date = []
    subject = []
    
    # Extract information from spaCy's entities and noun chunks
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'LOC':
            place.append(ent.text)
        elif ent.label_ == 'PERSON':
            subject.append(ent.text)
        elif ent.label_ == 'TIME':
            time.append(ent.text)
        elif ent.label_ == 'EVENT':
            event.append(ent.text)
        elif ent.label_ == 'DATE':
            date.append(ent.text)
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj' and chunk.root.head.pos_ == 'VERB':
            event.append(chunk.text)
        elif chunk.root.dep_ == 'pobj' and chunk.root.head.pos_ == 'ADP' and chunk.root.head.head.pos_ == 'VERB':
            place.append(chunk.text)
        elif chunk.root.dep_ == 'npadvmod' and chunk.root.head.pos_ == 'VERB':
            time.append(chunk.text)
        elif chunk.root.dep_ == 'pobj' and chunk.root.head.pos_ == 'ADP' and chunk.root.head.head.pos_ == 'VERB':
            date.append(chunk.text)
    
    # Create a dictionary to store the extracted information
    information = {'Place': place, 'Event': event, 'Time': time, 'Date': date, 'Subject': subject}
    
    print('Place:',place)
    print('Event:',event)
    print('Time:',time)
    print('Date:',date)
    print('Subject:',subject)


extract_information(text)

# Extract important details
for sent in doc.sents:
    for token in sent:
        if token.text == 'Keynote':
            print('Important things:', sent.text)




Place: ['Starbucks', '2pm']
Event: ["'s"]
Time: ['2pm']
Date: ['Friday']
Subject: []


# Text Msg

In [3]:
if not os.path.exists('/content/data/'):

    api_token = {"username": "gomteshjain12",
                 "key": "c1ce67cc586abf35c84ba89756ab5b72"}

    with open('/content/kaggle.json', 'w') as file:
        json.dump(api_token, file)

    os.environ["KAGGLE_CONFIG_DIR"] = "/content/"

    os.system('kaggle datasets download -d leizaaragon/emails-events')
    os.makedirs('/content/data/', exist_ok=True)
    os.system('mv /content/emails-events.zip /content/data/emails-events.zip')
    os.system('unzip -q /content/data/emails-events.zip -d /content/data/')
    os.remove('/content/data/emails-events.zip')
    

    

In [4]:
df= pd.read_csv("/content/data/emails_events.csv")

In [5]:
df.head()

,text,label
0,It's the first Monday after I walked out on my...,1
1,Our work schedule encompassed five intense day...,1
2,The highways were quiet on the Monday mid-morn...,1
3,"He left early each Friday afternoon, often ret...",1
4,The difference between her before her Monday n...,1


In [6]:
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.tag import pos_tag
# from nltk.chunk import ne_chunk
# from nltk.sentiment import SentimentIntensityAnalyzer
# from dateutil.parser import parse

In [7]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')
# nltk.download('vader_lexicon')

In [12]:
import pandas as pd
import spacy

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Define function to extract place, event, time, date, and subject from text
def extract_information(text):
    # Parse text with spaCy
    doc = nlp(text)
    
    # Initialize variables to store information
    place = []
    event = []
    time = []
    date = []
    subject = []
    
    # Extract information from spaCy's entities and noun chunks
    for ent in doc.ents:
        if ent.label_ == 'GPE' or ent.label_ == 'LOC':
            place.append(ent.text)
        elif ent.label_ == 'PERSON':
            subject.append(ent.text)
        elif ent.label_ == 'TIME':
            time.append(ent.text)
        elif ent.label_ == 'EVENT':
            event.append(ent.text)
        elif ent.label_ == 'DATE':
            date.append(ent.text)
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj' and chunk.root.head.pos_ == 'VERB':
            event.append(chunk.text)
        elif chunk.root.dep_ == 'pobj' and chunk.root.head.pos_ == 'ADP' and chunk.root.head.head.pos_ == 'VERB':
            place.append(chunk.text)
        elif chunk.root.dep_ == 'npadvmod' and chunk.root.head.pos_ == 'VERB':
            time.append(chunk.text)
        elif chunk.root.dep_ == 'pobj' and chunk.root.head.pos_ == 'ADP' and chunk.root.head.head.pos_ == 'VERB':
            date.append(chunk.text)
    
    # Create a dictionary to store the extracted information
    information = {'Place': place, 'Event': event, 'Time': time, 'Date': date, 'Subject': subject}
    
    return information

# Define function to create a DataFrame from a list of dictionaries
def create_dataframe(information_list):
    df = pd.DataFrame(information_list)
    return df

# Example usage
# text_messages = [
#     "John is going to the store at 2pm tomorrow to buy some groceries",
#     "I have a meeting with Sarah at the office on Friday",
#     "The concert is at the stadium tonight",
#     "Mary is having a party at her house on Saturday evening"
# ]

information_list = []
c=0
for text in df['text']:
    information = extract_information(text)
    information_list.append(information)
    if c%100 == 0:
      print(c)
    c=c+1
    
results = create_dataframe(information_list)
print(results)


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
                  Place                Event            

In [13]:
results.to_csv('text_event_final.csv')

#NER Spacy

In [53]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [54]:
!pip install jsonlines
!python -m spacy download en_core_web_lg
!python -m spacy download en_core_web_sm

!wget https://andrewhalterman.com/files/cleaned_masdar.jsonl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-05-01 19:17:21.222143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
2023-05-01 19:18:09.760566: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
--2023-05-01 19:18:20--  https://andrewhalterman.com/files/cleaned_masdar.jsonl
Resolving a

In [57]:
import jsonlines

from tqdm.autonotebook import tqdm
import jsonlines
import re

import spacy
from spacy import displacy
#assert spacy.__version__ == "3.1.3"


In [58]:
nlp = spacy.load("en_core_web_lg")
nlp_sm = spacy.load("en_core_web_sm")

In [59]:
with jsonlines.open("cleaned_masdar.jsonl", "r") as f:
    articles = list(f.iter())

print(len(articles))

7729


In [60]:
article = articles[313]
article

{'date': '13/02/2017',
 'body': 'BEIRUT, LEBANON (10:20 P.M.) – The Russian Air Force has launched several airstrikes over the eastern countryside of Aleppo tonight, hitting several of the Islamic State’s (ISIS) positions between Deir Hafer and Al-Bab.\nRussian jets traveled from the Hmaymim Military Airport in southwest Latakia to the Aleppo Governorate tonight in order to aid the Syrian and Turkish armies currently battling with the Islamic State forces in the Al-Bab Plateau and Deir Hafer Plain.\nAccording to local reports, Russian jets primarily focused on the\xa0\xa0road leading from Al-Bab to Deir Hafer; this area is where the Syrian Arab Army is currently attacking the Islamic State forces.\nRussian jets are still launching airstrikes this minute, forcing the Islamic State terrorists to to avoid launching counter-attacks against the Syrian and Turkish armies in east Aleppo.',
 'title': 'Russian jets hammer ISIS with nonstop airstrikes in east Aleppo'}

In [61]:
doc = nlp(article['body'])

In [62]:
# look document-level attributes
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_context',
 '_get_array_attrs',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'from_json',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set

In [63]:
# tokens in a document can by accessed by their number:
print(doc[5])
dir(doc[5])

P.M.


['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 'ancestors',
 'check_flag',
 'children',
 'cluster',
 'conjuncts',
 'dep',
 'dep_',
 'doc',
 'ent_id',
 'ent_id_',
 'ent_iob',
 'ent_iob_',
 'ent_kb_id',
 'ent_kb_id_',
 'ent_type',
 'ent_type_',
 'get_extension',
 'has_dep',
 'has_extension',
 'has_head',
 'has_morph',
 'has_vector',
 'head',
 'i',
 'idx',
 'iob_strings',
 'is_alpha',
 'is_ancestor',
 'is_ascii',
 'is_bracket',
 'is_currency',
 'is_digit',
 'is_left_punct',
 'is_lower',
 'is_oov',
 'is_punct',
 'is_quote',
 'is_right_punct',
 'is_sent_end',
 'is_sent_start',
 'is_space',
 'is_stop',
 'is_title',
 'is_upper',
 'lang',
 'lang_',
 'le

In [64]:
displacy.render(doc, style="ent", jupyter=True)

In [65]:
spacy.explain("GPE")

'Countries, cities, states'

In [66]:
[(i.text, i.ent_iob_ + "-" + i.ent_type_) for i in doc[0:30]]

[('BEIRUT', 'B-ORG'),
 (',', 'O-'),
 ('LEBANON', 'B-GPE'),
 ('(', 'O-'),
 ('10:20', 'B-TIME'),
 ('P.M.', 'I-TIME'),
 (')', 'O-'),
 ('–', 'O-'),
 ('The', 'B-ORG'),
 ('Russian', 'I-ORG'),
 ('Air', 'I-ORG'),
 ('Force', 'I-ORG'),
 ('has', 'O-'),
 ('launched', 'O-'),
 ('several', 'O-'),
 ('airstrikes', 'O-'),
 ('over', 'O-'),
 ('the', 'O-'),
 ('eastern', 'O-'),
 ('countryside', 'O-'),
 ('of', 'O-'),
 ('Aleppo', 'B-GPE'),
 ('tonight', 'B-TIME'),
 (',', 'O-'),
 ('hitting', 'O-'),
 ('several', 'O-'),
 ('of', 'O-'),
 ('the', 'B-ORG'),
 ('Islamic', 'I-ORG'),
 ('State', 'I-ORG')]

#Transformer-based spaCy model

In [68]:
!pip install --upgrade spacy[cuda111,transformers]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.4/100.4 MB 8.2 MB/s eta 0:00:00


In [69]:
!python -m spacy download en_core_web_trf

2023-05-01 19:29:46.492537: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


ValueError: ignored

In [71]:
!pip install spacy-transformers
import spacy_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
nlp_trf = spacy.load("en_core_web_trf")

In [73]:
doc = nlp_trf(articles[313]['body'])
displacy.render(doc, style="ent", jupyter=True)

In [88]:
for i in doc.ents:
  print(i.text,i.label_)

BEIRUT GPE
LEBANON GPE
10:20 P.M. TIME
The Russian Air Force ORG
Aleppo GPE
tonight TIME
the Islamic State’s ORG
ISIS ORG
Deir Hafer GPE
Al-Bab GPE
Russian NORP
the Hmaymim Military Airport FAC
Latakia GPE
Aleppo GPE
tonight TIME
Syrian NORP
Turkish NORP
Islamic State ORG
the Al-Bab Plateau LOC
Deir Hafer Plain LOC
Russian NORP
Al-Bab GPE
Deir Hafer GPE
the Syrian Arab Army ORG
Islamic State ORG
Russian NORP
this minute TIME
Islamic State ORG
Syrian NORP
Turkish NORP
Aleppo GPE


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.7 MB/s eta 0:00:00


In [3]:
import torch
from transformers import pipeline

# Load NER model
nlp = pipeline("ner")

# Sample text
text = "The conference will be held on July 15th, 2023 in New York City."

# Extract named entities
entities = nlp(text)

# Print the extracted entities
for entity in entities:
    print(entity)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


{'entity': 'I-LOC', 'score': 0.999584, 'index': 13, 'word': 'New', 'start': 50, 'end': 53}
{'entity': 'I-LOC', 'score': 0.99949, 'index': 14, 'word': 'York', 'start': 54, 'end': 58}
{'entity': 'I-LOC', 'score': 0.9996025, 'index': 15, 'word': 'City', 'start': 59, 'end': 63}
